In [2]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath("../../mathematical_models"))
from mathematical_models.s_on_f import ScalarOnFunctionModel
from mathematical_models.f_on_f import FunctionOnFunctionModel
from mathematical_models.s_on_s import ScalarOnScalarModel

sys.path.append(os.path.abspath("../../optimizers"))
from optimizers.nbdo import NBDO

sys.path.append(os.path.abspath("../../utilities"))
from utilities.plotting.plot_fun import subplot_results

### Settings

In [3]:
N = 24 # runs
Kx = [[5]]
epochs = 100
order = 1
s_on_s_model = ScalarOnScalarModel(Kx=Kx[0], order=order)

In [4]:
optimizer_s_on_s = NBDO(model=s_on_s_model, latent_dim=4)
optimizer_s_on_s.compute_train_set(num_designs=1_000, runs=N)
history = optimizer_s_on_s.fit(epochs=epochs)

Epoch 1/100


2024-09-04 19:23:57.870536: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


OperatorNotAllowedInGraphError: in user code:

    File "/Users/thodorisladas/miniforge3/envs/optidex/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/thodorisladas/Documents/code/system_optidex/optimizers/nbdo.py", line 162, in custom_loss  *
        objective_value = self.model.compute_objective_tf(y_pred, m, n)
    File "/Users/thodorisladas/Documents/code/system_optidex/mathematical_models/s_on_s.py", line 73, in compute_objective_tf  *
        Z_t_Z = self.calc_covar_matrix_tf(X)
    File "/Users/thodorisladas/Documents/code/system_optidex/mathematical_models/s_on_s.py", line 67, in calc_covar_matrix_tf  *
        model_mat = self.calc_model_matrix_tf(Model_mat)
    File "/Users/thodorisladas/Documents/code/system_optidex/mathematical_models/s_on_s.py", line 60, in calc_model_matrix_tf  *
        for combo in combinations_with_replacement(range(n_features), o):

    OperatorNotAllo

In [5]:
best_cr, best_des = optimizer_s_on_s.optimize()

Iteration No: 1 started. Evaluating function at random point.
1/1 [==============================] - 0s 59ms/step
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0979
Function value obtained: 7.0236
Current minimum: 7.0236
Iteration No: 2 started. Evaluating function at random point.
1/1 [==============================] - 0s 9ms/step
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0245
Function value obtained: 7.0397
Current minimum: 7.0236
Iteration No: 3 started. Evaluating function at random point.
1/1 [==============================] - 0s 8ms/step
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0229
Function value obtained: 5.4473
Current minimum: 5.4473
Iteration No: 4 started. Evaluating function at random point.
1/1 [==============================] - 0s 12ms/step
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0256
Function value obtained: 3.5725
Current minimum: 3.5725
Iteration No: 5 starte

In [7]:
print(best_des)
# 0.2551

[[ 0.02441331  0.26136    -0.33270615 -0.17181681 -0.05859732]
 [-0.50537074  0.18516193  0.03489427 -0.6624551   0.34770757]
 [ 0.5130446   0.19772674 -0.12861443  0.15613754  0.22907995]
 [ 0.26292562 -0.11875507  0.07083628  0.15033314  0.46356642]
 [ 0.30730212 -0.28946683  0.13278474 -0.17912658  0.04701832]
 [-0.7156744  -0.00716581  0.4342425  -0.26347947 -0.2260786 ]
 [-0.19682075 -0.649097    0.16571647  0.11571792  0.13326521]
 [-0.04229975 -0.09577472  0.24881646 -0.30867484 -0.28512177]
 [-0.13734275  0.41831517  0.28368285 -0.3132838   0.3840341 ]
 [ 0.08650683 -0.0789809  -0.3626668   0.04341892  0.35813966]
 [-0.01147771 -0.11277858  0.2275215   0.7034796   0.52255404]
 [ 0.37203124 -0.3900067   0.35489482 -0.7561665   0.80261904]
 [-0.2446713  -0.10171866  0.28816348 -0.2600224   0.32948515]
 [ 0.41865218 -0.8487816   0.24245083 -0.49949655  0.46177745]
 [ 0.32041252 -0.06853209  0.74928445  0.68706733 -0.21938625]
 [ 0.45167434 -0.01155688  0.37207142  0.14272648 -0.50

In [8]:
best_cr

2.419059936128866

In [8]:
print(np.round(best_des.T @ best_des))

[[24.  0.  0. -0. -2.]
 [ 0. 24.  0.  4. -2.]
 [ 0.  0. 24.  0.  2.]
 [-0.  4.  0. 24. -2.]
 [-2. -2.  2. -2. 24.]]


In [1]:
import tensorflow as tf

def compute_objective_tf(self, X, m, n):
    batch_size = tf.shape(X)[0]
    ones = tf.ones((batch_size, m, 1))
    X = tf.reshape(X, (-1, m, n))
    Z = tf.concat([ones, tf.matmul(X, self.J_cb)], axis=2)

    Z_t_Z = tf.matmul(Z, Z, transpose_a=True)
    det = tf.linalg.det(Z_t_Z)
    epsilon = 1e-6
    condition = tf.abs(det)[:, None, None] < epsilon

    diagonal = tf.linalg.diag_part(Z_t_Z) + epsilon
    Z_t_Z_epsilon = Z_t_Z + tf.linalg.diag(diagonal - tf.linalg.diag_part(Z_t_Z))
    Z_t_Z_regularized = tf.where(condition, Z_t_Z_epsilon, Z_t_Z)

    M = tf.linalg.inv(Z_t_Z_regularized)
    value = tf.linalg.trace(M)
    return tf.where(value < 0, tf.constant(1e10), value)
